In [ ]:
from IPython import get_ipython
from IPython.display import display

# Installer les dépendances nécessaires
!pip install -U langgraph langsmith langchain_anthropic
!pip install -U openai
!pip install -U transformers huggingface_hub
!pip install torch

import os

# Charger le token Hugging Face depuis .env
from google.colab import files
uploaded = files.upload()  # Téléchargez votre fichier .env

from dotenv import load_dotenv
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM  # Import manquant

load_dotenv(".env")  # Charge votre token
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
print("✅ Token chargé :", token[:10] + "...")

# Charger le modèle Mistral
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Charger le tokenizer et le modèle avec float16
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=token,
    torch_dtype=torch.float16,  # Forcer FP16 pour réduire la mémoire
    device_map="auto"  # GPU si disponible
)

# Vérifier le chargement du modèle
print("✅ Modèle chargé sur :", model.device)

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from transformers import AutoTokenizer
import textwrap
import random

prompts = [
    "Quels sont les principaux avantages de l'IA pour la société?",
    "Quels sont les dangers potentiels de l'IA?",
    "Comment l'IA peut-elle être utilisée pour résoudre des problèmes mondiaux?",
    "L'IA est-elle une menace pour l'emploi?",
    "Comment garantir une utilisation éthique de l'IA?",
]

# Définition de l'état du débat
class DebateState(TypedDict):
    messages: Annotated[list, add_messages]  # Liste des messages échangés

# Création du graphe
graph_builder = StateGraph(DebateState)

def agent1(state: DebateState):
    """L'agent 1 donne son point de vue"""
    last_message = state["messages"][-1].content
    # Extraire le prompt aléatoire du dernier message
    last_prompt = last_message.split(" ")[-1]

    # Formuler l'input pour le modèle en fonction du prompt
    input_text = f"## Argument en faveur de l'IA concernant: {last_prompt}\n"  # Début de l'argument

    # ... (Vous pouvez ajouter des instructions spécifiques ici)
    input_text += "Veuillez développer un argument en faveur des bénéfices de l'IA en vous concentrant sur ce point."

    # Définir 'inputs'
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=150)

    response_text = tokenizer.decode(output[0], skip_special_tokens=True)
    response_text = textwrap.fill(response_text, width=80)
    return {"messages": state["messages"] + [AIMessage(content=response_text, type='ai')]}

def agent2(state: DebateState):
    """L'agent 2 réagit à l'argument de l'agent 1"""
    last_message = state["messages"][-1].content
    last_prompt = last_message.split(" ")[-1]  # Supposant que le prompt est le dernier mot

    # Formuler l'input pour le modèle en fonction du prompt
    input_text = f"## Argument contre l'IA concernant: {last_prompt}\n"

    # ... (Vous pouvez ajouter des instructions spécifiques ici)
    input_text += "Veuillez développer un argument contre les dangers de l'IA en vous concentrant sur ce point."

    # Définir 'inputs'
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)


    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=150, temperature=0.8)

    response_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Ajuster le retour à la ligne du texte
    response_text = textwrap.fill(response_text, width=80)  # Ajuster la largeur selon vos besoins
    # Retourner un AIMessage
    return {"messages": state["messages"] + [AIMessage(content=response_text, type='ai')]}

# Ajouter les agents dans le graphe
graph_builder.add_node("agent1", agent1)
graph_builder.add_node("agent2", agent2)

# Lancement du débat par l'agent 1
graph_builder.add_edge(START, "agent1")
# L'agent 1 passe la main à l'agent 2
graph_builder.add_edge("agent1", "agent2")
# L'agent 2 renvoie à l'agent 1 pour continuer le débat
graph_builder.add_edge("agent2", "agent1")

In [ ]:
# Compiler le graphe
debate_graph = graph_builder.compile()

# Fonction pour simuler le débat
def run_debate(initial_prompt, num_turns=5):
    # Initialiser avec HumanMessage
    state = {"messages": [HumanMessage(content=initial_prompt)]}

    for i, event in enumerate(debate_graph.stream(state)):
       # Choisir un prompt aléatoire pour ce tour
        current_prompt = random.choice(prompts)

        # Ajouter le prompt au message du tour actuel
        state['messages'][-1].content += " " + current_prompt # on ajoute le prompt current au dernier prompt existant

        for value in event.values():
            # Convertir le dernier message en AIMessage ou HumanMessage
            last_message = value['messages'][-1]
            # Accéder au type du message avec .type
            if last_message.type in ("agent1", "agent2"):
                message = AIMessage(content=last_message.content)
            else:
                message = HumanMessage(content=last_message.content)

            print(f"🗣️ Tour {i+1} - {message.type} : {message.content}")

        if i >= num_turns - 1:
            break


# Lancer le débat avec un sujet donné
run_debate("L'intelligence artificielle est-elle bénéfique ou dangereuse ?")